# Librerie e Funzioni

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, TensorDataset, Dataset
from torch import optim, nn
from torch.nn import functional
import os
import time
import csv
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import shutil
from torchvision.utils import make_grid
from random import randint
from PIL import Image
import random

from einops import rearrange
from einops.layers.torch import Rearrange

import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from vit_pytorch.ats_vit import ViT as ATSViT
from vit_pytorch import ViT

from MultiViT import MultiViT
from TopK import TopKViT as ViT_topk
from PatchMergerViT import PatchMergerViT
from ToMe import ToMe
from EViT import EViT
from DWTViT import DWTViT
from DWTViT_pruning import DWTViT_pruning
from DWTViT_quantile import DWTViT_quantile
from DWTViT_gini import DWTViT_gini


from tqdm import tqdm

import torch
from torch import nn

from einops import rearrange, repeat
from einops.layers.torch import Rearrange

from thop import profile

import torch
import time

/home/dtraini/.local/lib/python3.10/site-packages/pytorch_wavelets/dtcwt/coeffs.py:7: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_stream


## Benchmark

In [2]:
def benchmark_time(
    model,
    device,
    input_size,
    batch_size,
    runs,
    throw_out,
    verbose,
):
    """
    Benchmark the given model with random inputs at the given batch size.

    Args:
     - model: the module to benchmark
     - device: the device to use for benchmarking
     - input_size: the input size to pass to the model (channels, h, w)
     - batch_size: the batch size to use for evaluation
     - runs: the number of total runs to do
     - throw_out: the percentage of runs to throw out at the start of testing
     - verbose: whether or not to use tqdm to print progress / print throughput at end

    Returns:
     - the throughput measured in images / second
    """
    if not isinstance(device, torch.device):
        device = torch.device(device)
    is_cuda = torch.device(device).type == "cuda"

    model = model.eval().to(device)
    print(device)
    input = torch.rand(batch_size, *input_size, device=device)

    warm_up = int(runs * throw_out)
    total = 0
    start = time.time()

    with torch.autocast(device.type):
        with torch.no_grad():
            for i in tqdm(range(runs), disable=not verbose, desc="Benchmarking"):
                if i == warm_up:
                    if is_cuda:
                        torch.cuda.synchronize()
                    total = 0
                    start = time.time()

                model(input)
                total += batch_size

    if is_cuda:
        torch.cuda.synchronize()

    end = time.time()
    elapsed = end - start

    throughput = total / elapsed
    throughput = f'{throughput:.2f}'

    if verbose:
        print(f"Throughput: {throughput:.2f} im/s")

    return [throughput]





def benchmark_flops(
    model,
    device,
    input_size,
    batch_size,
    runs,
    verbose,
):
    """
    Benchmark the given model with random inputs at the given batch size.

    Args:
     - model: the module to benchmark
     - device: the device to use for benchmarking
     - input_size: the input size to pass to the model (channels, h, w)
     - batch_size: the batch size to use for evaluation
     - runs: the number of total runs to do
     - verbose: whether or not to use tqdm to print progress / print throughput at end

    Returns:
     - the throughput measured in images / second
    """
    if not isinstance(device, torch.device):
        device = torch.device(device)
    is_cuda = torch.device(device).type == "cuda"

    model = model.eval().to(device)
    print(device)
    input = torch.rand(batch_size, *input_size, device=device)

    total = 0
    total_flops = 0
    total_params = 0

    with torch.autocast(device.type):
        with torch.no_grad():
            for i in tqdm(range(runs), disable=not verbose, desc="Benchmarking"):
                flops, params = profile(model, inputs=(input, ), verbose=False)
                total += batch_size
                total_flops += flops

    flops = total_flops/total
    flops = f'{flops/1e9:.4f}'
    if verbose:
        print(f"Total FLOPs: {flops / 1e9:.4f} GFLOPs")
        print(f"Total Parameters: {params / 1e6:.4f} M")

    return [flops], [params]



def create_patch_list(total_patches, cut):
    # Calcola l'importo scontato per ogni blocco di 3 elementi
    discounted_patches = total_patches
    patch_list = []
    for i in range(12):
        if i % 3 == 0 and i != 0:
            discounted_patches = int(discounted_patches * (cut / 100))
        patch_list.append(discounted_patches)
    return patch_list

In [3]:
def get_results(batch_size, patch_size, runs_time, runs_flops, input_size, img_size, att_dim, depth, heads, mlp_dim, max_tokens_per_depth, patch_merge_layers, tome_patches, DW_patches, type, df, device):

    # ViT
    print('##### ViT ####')
    ViTnet = ViT(
        image_size = img_size,
        patch_size = patch_size,
        num_classes = 10,
        dim = att_dim,
        depth = depth,
        heads = heads,
        mlp_dim = mlp_dim,
    )

    ViTnet.to(device)

    baseline_throughput = benchmark_time(
        ViTnet,
        device=device,
        verbose=False,
        runs=runs_time,
        batch_size=batch_size,
        input_size=input_size,
        throw_out = 0.25
    )

    baseline_flops, baseline_params = benchmark_flops(
        ViTnet,
        device=device,
        verbose=False,
        runs=runs_time,
        batch_size=batch_size,
        input_size=input_size
    )

    df = pd.concat([df, pd.DataFrame({'Modello': f'ViT_net_{type}', 'Throughput': baseline_throughput, 'Flops': baseline_flops, 'Params': baseline_params})], ignore_index=True)

    # ATS
    print('##### ATSViT ####')
    ATSViTnet = ATSViT(
        image_size = img_size,
        patch_size = patch_size,
        num_classes = 10,
        dim = att_dim,
        depth = depth,
        heads = heads,
        mlp_dim = mlp_dim,
        max_tokens_per_depth = max_tokens_per_depth
    )

    ATSViTnet.to(device)

    ATS_throughput = benchmark_time(
        ATSViTnet,
        device=device,
        verbose=False,
        runs=runs_time,
        batch_size=batch_size,
        input_size=input_size,
        throw_out = 0.25
    )

    ATS_flops, ATS_params = benchmark_flops(
        ATSViTnet,
        device=device,
        verbose=False,
        runs=runs_time,
        batch_size=batch_size,
        input_size=input_size
    )

    df = pd.concat([df, pd.DataFrame({'Modello': f'ATS_{type}', 'Throughput': ATS_throughput, 'Flops': ATS_flops, 'Params': ATS_params})], ignore_index=True)

    # MultiViT
    print('##### MultiViT ####')
    MultiViTnet = MultiViT(
        image_size = img_size,
        patch_size = patch_size,
        num_classes = 10,
        dim = att_dim,
        depth = depth,
        heads = heads,
        mlp_dim = mlp_dim,
        n_patch = max_tokens_per_depth
    )

    MultiViTnet.to(device)


    MultiViT_throughput = benchmark_time(
        MultiViTnet,
        device=device,
        verbose=False,
        runs=runs_time,
        batch_size=batch_size,
        input_size=input_size,
        throw_out = 0.25
    )

    MultiViT_flops, MultiViT_params = benchmark_flops(
        MultiViTnet,
        device=device,
        verbose=False,
        runs=runs_time,
        batch_size=batch_size,
        input_size=input_size
    )

    df = pd.concat([df, pd.DataFrame({'Modello': f'MultiViT_{type}', 'Throughput': MultiViT_throughput, 'Flops': MultiViT_flops, 'Params': MultiViT_params})], ignore_index=True)


    print('##### Transformer_topk ####')
    # Transformer_topk
    Transformer_topknet = ViT_topk(
        image_size = img_size,
        patch_size = patch_size,
        num_classes = 10,
        dim = att_dim,
        depth = depth,
        heads = heads,
        mlp_dim = mlp_dim,
        n_patch = max_tokens_per_depth
    )

    Transformer_topknet.to(device)

    Transformer_topk_throughput = benchmark_time(
        Transformer_topknet,
        device=device,
        verbose=False,
        runs=runs_time,
        batch_size=batch_size,
        input_size=input_size,
        throw_out = 0.25
    )

    Transformer_topk_flops, Transformer_topk_params = benchmark_flops(
        Transformer_topknet,
        device=device,
        verbose=False,
        runs=runs_time,
        batch_size=batch_size,
        input_size=input_size
    )

    df = pd.concat([df, pd.DataFrame({'Modello': f'Transformer_topk_{type}', 'Throughput': Transformer_topk_throughput, 'Flops': Transformer_topk_flops, 'Params': Transformer_topk_params})], ignore_index=True)



    print('##### Transformer_evit ####')
    Transformer_evitnet = EViT(
        image_size=img_size,
        patch_size=patch_size,
        num_classes=10,
        dim=att_dim,
        depth=depth,
        heads=heads,
        mlp_dim=mlp_dim,
        n_patch = max_tokens_per_depth
    )

    Transformer_evitnet.to(device)

    Transformer_evit_throughput = benchmark_time(
        Transformer_evitnet,
        device=device,
        verbose=False,
        runs=runs_time,
        batch_size=batch_size,
        input_size=input_size,
        throw_out = 0.25
    )

    Transformer_evit_flops, Transformer_evit_params = benchmark_flops(
        Transformer_evitnet,
        device=device,
        verbose=False,
        runs=runs_time,
        batch_size=batch_size,
        input_size=input_size
    )

    df = pd.concat([df, pd.DataFrame({'Modello': f'Transformer_evit_{type}', 'Throughput': Transformer_evit_throughput, 'Flops': Transformer_evit_flops, 'Params': Transformer_evit_params})], ignore_index=True)


    print('##### Transformer_tome ####')
    Transformer_tomenet = ToMe(
        image_size=img_size,
        patch_size=patch_size,
        num_classes=10,
        dim=att_dim,
        depth=depth,
        heads=heads,
        mlp_dim=mlp_dim,
        n_patch = tome_patches
    )

    Transformer_tomenet.to(device)

    Transformer_tome_throughput = benchmark_time(
        Transformer_tomenet,
        device=device,
        verbose=False,
        runs=runs_time,
        batch_size=batch_size,
        input_size=input_size,
        throw_out = 0.25
    )

    Transformer_tome_flops, Transformer_tome_params = benchmark_flops(
        Transformer_tomenet,
        device=device,
        verbose=False,
        runs=runs_time,
        batch_size=batch_size,
        input_size=input_size
    )

    df = pd.concat([df, pd.DataFrame({'Modello': f'Transformer_tome_{type}', 'Throughput': Transformer_tome_throughput, 'Flops': Transformer_tome_flops, 'Params': Transformer_tome_params})], ignore_index=True)

    for wavelet in ['haar', 'sym2', 'db4', 'db2']:
            print('##### Transformer_DWTViT ####')
            Transformer_DWTViT = DWTViT_pruning(
                image_size=img_size,
                patch_size=patch_size,
                num_classes=10,
                dim=att_dim,
                depth=depth,
                heads=heads,
                mlp_dim=mlp_dim,
                wavelet=wavelet,
                pruning_locations = DW_patches
            )
        
            Transformer_DWTViT.to(device)
        
            Transformer_DWTViT_throughput = benchmark_time(
                Transformer_DWTViT,
                device=device,
                verbose=False,
                runs=runs_time,
                batch_size=batch_size,
                input_size=input_size,
                throw_out = 0.25
            )
        
            Transformer_DWTViT_flops, Transformer_DWTViT_params = benchmark_flops(
                Transformer_DWTViT,
                device=device,
                verbose=False,
                runs=runs_time,
                batch_size=batch_size,
                input_size=input_size
            )
        
        
            df = pd.concat([df, pd.DataFrame({'Modello': f'Transformer_DWTViT_{wavelet}_{type}', 'Throughput': Transformer_DWTViT_throughput, 'Flops': Transformer_DWTViT_flops, 'Params': Transformer_DWTViT_params})], ignore_index=True)


    for strategy in ['cA', 'cD']:
            print('##### Transformer_DWTViT ####')
            Transformer_DWTViT = DWTViT_pruning(
                image_size=img_size,
                patch_size=patch_size,
                num_classes=10,
                dim=att_dim,
                depth=depth,
                heads=heads,
                mlp_dim=mlp_dim,
                wavelet='haar',
                pruning_locations = DW_patches,
                strategy = strategy
            )
        
            Transformer_DWTViT.to(device)
        
            Transformer_DWTViT_throughput = benchmark_time(
                Transformer_DWTViT,
                device=device,
                verbose=False,
                runs=runs_time,
                batch_size=batch_size,
                input_size=input_size,
                throw_out = 0.25
            )
        
            Transformer_DWTViT_flops, Transformer_DWTViT_params = benchmark_flops(
                Transformer_DWTViT,
                device=device,
                verbose=False,
                runs=runs_time,
                batch_size=batch_size,
                input_size=input_size
            )
        
        
            df = pd.concat([df, pd.DataFrame({'Modello': f'Transformer_DWTViT_{strategy}_{type}', 'Throughput': Transformer_DWTViT_throughput, 'Flops': Transformer_DWTViT_flops, 'Params': Transformer_DWTViT_params})], ignore_index=True)


    print('##### Transformer_DWTViT_pruning ####')
    Transformer_DWTViT_pruning = DWTViT_pruning(
        image_size=img_size,
        patch_size=patch_size,
        num_classes=10,
        dim=att_dim,
        depth=depth,
        heads=heads,
        mlp_dim=mlp_dim,
        wavelet='db4',
        pruning_locations=DW_patches,
        # pruning_aggressiveness=1,
    )

    Transformer_DWTViT_pruning.to(device)

    Transformer_DWTViT_pruning_throughput = benchmark_time(
        Transformer_DWTViT_pruning,
        device=device,
        verbose=False,
        runs=runs_time,
        batch_size=batch_size,
        input_size=input_size,
        throw_out = 0.25
    )

    Transformer_DWTViT_pruning_flops, Transformer_DWTViT_pruning_params = benchmark_flops(
        Transformer_DWTViT_pruning,
        device=device,
        verbose=False,
        runs=runs_time,
        batch_size=batch_size,
        input_size=input_size
    )


    df = pd.concat([df, pd.DataFrame({'Modello': f'Transformer_DWTViT_pruning_{type}', 'Throughput': Transformer_DWTViT_pruning_throughput, 'Flops': Transformer_DWTViT_pruning_flops, 'Params': Transformer_DWTViT_pruning_params})], ignore_index=True)



    print('##### Transformer_DWTViT_gini ####')
    Transformer_DWTViT_gini = DWTViT_gini(
        image_size=img_size,
        patch_size=patch_size,
        num_classes=10,
        dim=att_dim,
        depth=depth,
        heads=heads,
        mlp_dim=mlp_dim,
        wavelet='haar',
        pruning_locations=DW_patches,
        soglia = 0.5
    )

    Transformer_DWTViT_gini.to(device)

    Transformer_DWTViT_gini_throughput = benchmark_time(
        Transformer_DWTViT_gini,
        device=device,
        verbose=False,
        runs=runs_time,
        batch_size=batch_size,
        input_size=input_size,
        throw_out = 0.25
    )

    Transformer_DWTViT_gini_flops, Transformer_DWTViT_gini_params = benchmark_flops(
        Transformer_DWTViT_gini,
        device=device,
        verbose=False,
        runs=runs_time,
        batch_size=batch_size,
        input_size=input_size
    )


    df = pd.concat([df, pd.DataFrame({'Modello': f'Transformer_DWTViT_gini_{type}', 'Throughput': Transformer_DWTViT_gini_throughput, 'Flops': Transformer_DWTViT_gini_flops, 'Params': Transformer_DWTViT_gini_params})], ignore_index=True)



    print('##### Transformer_DWTViT_quantile ####')
    Transformer_DWTViT_quantile = DWTViT_quantile(
        image_size=img_size,
        patch_size=patch_size,
        num_classes=10,
        dim=att_dim,
        depth=depth,
        heads=heads,
        mlp_dim=mlp_dim,
        wavelet='haar',
        pruning_locations=DW_patches,
    )

    Transformer_DWTViT_quantile.to(device)

    Transformer_DWTViT_quantile_throughput = benchmark_time(
        Transformer_DWTViT_quantile,
        device=device,
        verbose=False,
        runs=runs_time,
        batch_size=batch_size,
        input_size=input_size,
        throw_out = 0.25
    )

    Transformer_DWTViT_quantile_flops, Transformer_DWTViT_quantile_params = benchmark_flops(
        Transformer_DWTViT_quantile,
        device=device,
        verbose=False,
        runs=runs_time,
        batch_size=batch_size,
        input_size=input_size
    )


    df = pd.concat([df, pd.DataFrame({'Modello': f'Transformer_DWTViT_quantile_{type}', 'Throughput': Transformer_DWTViT_quantile_throughput, 'Flops': Transformer_DWTViT_quantile_flops, 'Params': Transformer_DWTViT_quantile_params})], ignore_index=True)

    
    # Patch merger
    print('##### PatchMerger ####')
    PatchMergerViTnet = PatchMergerViT(
        image_size = img_size,
        patch_size = patch_size,
        num_classes = 10,
        dim = att_dim,
        depth = depth,
        heads = heads,
        mlp_dim = mlp_dim,
        patch_merge_layers = patch_merge_layers
    )

    PatchMergerViTnet.to(device)


    PatchMerger_throughput = benchmark_time(
        PatchMergerViTnet,
        device=device,
        verbose=False,
        runs=runs_time,
        batch_size=batch_size,
        input_size=input_size,
        throw_out = 0.25
    )


    PatchMerger_flops, PatchMerger_params = benchmark_flops(
        PatchMergerViTnet,
        device=device,
        verbose=False,
        runs=runs_time,
        batch_size=batch_size,
        input_size=input_size
    )


    df = pd.concat([df, pd.DataFrame({'Modello': f'PatchMerger_{type}', 'Throughput': PatchMerger_throughput, 'Flops': PatchMerger_flops, 'Params': PatchMerger_params})], ignore_index=True)

    return df

In [4]:
def create_patch_list_reversed(total_patches, cut):
    remaining_patches = total_patches
    removal_list = []
    
    for i in range(12):
        if i % 3 == 0 and i != 0:
            removed_patches = remaining_patches - int(remaining_patches * (cut / 100))
            remaining_patches -= removed_patches
        else:
            removed_patches = 0
        
        removal_list.append(removed_patches)
    
    return removal_list

# Creazione tabella

## GPU

In [5]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
# device = torch.device('cpu')

img_size = 160
batch_size = 64
patch_size = 16
runs_time = 10
runs_flops = 10
input_size = (3, img_size, img_size)

DW_patches = [4, 6, 8]

att_dim = 768
depth = 12
heads = 12
mlp_dim = att_dim * 4

total_patches = int((img_size/patch_size)**2)
cut = 75
max_tokens_per_depth = create_patch_list(total_patches, cut)

patch_merge_layers = [(2, max_tokens_per_depth[3]),(5, max_tokens_per_depth[6]),(8, max_tokens_per_depth[9])] 

tome_patches = create_patch_list_reversed(total_patches, cut)

type = 'Base'

df_gpu = pd.DataFrame(columns = ['Modello', 'Throughput', 'Flops', 'Params'])

In [6]:
df_gpu = get_results(batch_size, patch_size, runs_time, runs_flops, input_size, img_size, att_dim, depth, heads, mlp_dim, max_tokens_per_depth, patch_merge_layers, tome_patches, DW_patches, type, df_gpu, device)

##### ViT ####
cuda:0
cuda:0
##### ATSViT ####
cuda:0
cuda:0
##### MultiViT ####
cuda:0
cuda:0
##### Transformer_topk ####
cuda:0
cuda:0
##### Transformer_evit ####
cuda:0
cuda:0
##### Transformer_tome ####
cuda:0
cuda:0
##### Transformer_DWTViT ####
cuda:0
cuda:0
##### Transformer_DWTViT ####
cuda:0
cuda:0
##### Transformer_DWTViT ####
cuda:0
cuda:0
##### Transformer_DWTViT ####
cuda:0
cuda:0
##### Transformer_DWTViT ####
cuda:0
cuda:0
##### Transformer_DWTViT ####
cuda:0
cuda:0
##### Transformer_DWTViT_pruning ####
cuda:0
cuda:0
##### Transformer_DWTViT_gini ####
cuda:0
cuda:0
##### Transformer_DWTViT_quantile ####
cuda:0
cuda:0
##### PatchMerger ####
cuda:0
cuda:0


In [7]:
att_dim = 384
depth = 12
heads = 6
mlp_dim = att_dim * 4

type = 'Small'

df_small_gpu = pd.DataFrame(columns = ['Modello', 'Throughput', 'Flops', 'Params'])

In [8]:
df_small_gpu = get_results(batch_size, patch_size, runs_time, runs_flops, input_size, img_size, att_dim, depth, heads, mlp_dim, max_tokens_per_depth, patch_merge_layers, tome_patches, DW_patches, type, df_small_gpu, device)

##### ViT ####
cuda:0
cuda:0
##### ATSViT ####
cuda:0
cuda:0
##### MultiViT ####
cuda:0
cuda:0
##### Transformer_topk ####
cuda:0
cuda:0
##### Transformer_evit ####
cuda:0
cuda:0
##### Transformer_tome ####
cuda:0
cuda:0
##### Transformer_DWTViT ####
cuda:0
cuda:0
##### Transformer_DWTViT ####
cuda:0
cuda:0
##### Transformer_DWTViT ####
cuda:0
cuda:0
##### Transformer_DWTViT ####
cuda:0
cuda:0
##### Transformer_DWTViT ####
cuda:0
cuda:0
##### Transformer_DWTViT ####
cuda:0
cuda:0
##### Transformer_DWTViT_pruning ####
cuda:0
cuda:0
##### Transformer_DWTViT_gini ####
cuda:0
cuda:0
##### Transformer_DWTViT_quantile ####
cuda:0
cuda:0
##### PatchMerger ####
cuda:0
cuda:0


In [9]:
att_dim = 192
depth = 12
heads = 3
mlp_dim = att_dim * 4

type = 'Tiny'

df_tiny_gpu = pd.DataFrame(columns = ['Modello', 'Throughput', 'Flops', 'Params'])

In [10]:
df_tiny_gpu = get_results(batch_size, patch_size, runs_time, runs_flops, input_size, img_size, att_dim, depth, heads, mlp_dim, max_tokens_per_depth, patch_merge_layers, tome_patches, DW_patches, type, df_tiny_gpu, device)

##### ViT ####
cuda:0
cuda:0
##### ATSViT ####
cuda:0
cuda:0
##### MultiViT ####
cuda:0
cuda:0
##### Transformer_topk ####
cuda:0
cuda:0
##### Transformer_evit ####
cuda:0
cuda:0
##### Transformer_tome ####
cuda:0
cuda:0
##### Transformer_DWTViT ####
cuda:0
cuda:0
##### Transformer_DWTViT ####
cuda:0
cuda:0
##### Transformer_DWTViT ####
cuda:0
cuda:0
##### Transformer_DWTViT ####
cuda:0
cuda:0
##### Transformer_DWTViT ####
cuda:0
cuda:0
##### Transformer_DWTViT ####
cuda:0
cuda:0
##### Transformer_DWTViT_pruning ####
cuda:0
cuda:0
##### Transformer_DWTViT_gini ####
cuda:0
cuda:0
##### Transformer_DWTViT_quantile ####
cuda:0
cuda:0
##### PatchMerger ####
cuda:0
cuda:0


### Display

In [11]:
display(df_gpu), display(df_small_gpu), display(df_tiny_gpu)

,Modello,Throughput,Flops,Params
0,ViT_net_Base,793.98,8.6502,85629706.0
1,ATS_Base,931.37,5.3217,85629706.0
2,MultiViT_Base,1027.11,6.0855,85626634.0
3,Transformer_topk_Base,1015.16,6.0855,85626634.0
4,Transformer_evit_Base,1029.55,6.1682,85626634.0
5,Transformer_tome_Base,1024.95,6.0855,85626634.0
6,Transformer_DWTViT_haar_Base,1144.23,4.6162,85626634.0
7,Transformer_DWTViT_sym2_Base,1117.55,4.7107,85626634.0
8,Transformer_DWTViT_db4_Base,1070.08,4.8595,85626634.0
9,Transformer_DWTViT_db2_Base,1119.48,4.7107,85626634.0


,Modello,Throughput,Flops,Params
0,ViT_net_Small,2026.76,2.1806,21581962.0
1,ATS_Small,1718.16,1.3545,21581962.0
2,MultiViT_Small,2472.32,1.5381,21579658.0
3,Transformer_topk_Small,2472.74,1.5381,21579658.0
4,Transformer_evit_Small,2496.32,1.5588,21579658.0
5,Transformer_tome_Small,2496.37,1.5381,21579658.0
6,Transformer_DWTViT_haar_Small,2412.40,1.1703,21579658.0
7,Transformer_DWTViT_sym2_Small,2334.14,1.1940,21579658.0
8,Transformer_DWTViT_db4_Small,2308.58,1.2312,21579658.0
9,Transformer_DWTViT_db2_Small,2391.03,1.1940,21579658.0


,Modello,Throughput,Flops,Params
0,ViT_net_Tiny,4091.76,0.5543,5483338.0
1,ATS_Tiny,2252.83,0.3444,5483338.0
2,MultiViT_Tiny,5053.16,0.3929,5481418.0
3,Transformer_topk_Tiny,5100.24,0.3929,5481418.0
4,Transformer_evit_Tiny,5414.91,0.3981,5481418.0
5,Transformer_tome_Tiny,5193.41,0.3929,5481418.0
6,Transformer_DWTViT_haar_Tiny,3530.08,0.3007,5481418.0
7,Transformer_DWTViT_sym2_Tiny,3400.96,0.3067,5481418.0
8,Transformer_DWTViT_db4_Tiny,3308.14,0.3160,5481418.0
9,Transformer_DWTViT_db2_Tiny,3206.66,0.3067,5481418.0


(None, None, None)